In [1]:
import numpy as np
import random
import librosa
import matplotlib.pyplot as plt
from IPython.display import Audio
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from data import BirdClefTrainAudio, BirdClefHarmonics
from model import HarmonicModel
from mimir import training
import time
import pickle
import os

In [2]:
np.random.seed(20250403)
random.seed(20250403)
torch.manual_seed(20250403)
torch.cuda.manual_seed(20250403)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [3]:
data_folder = "data"

In [4]:
audio = BirdClefTrainAudio(data_folder, max_duration=5, sr=16000)
cachefile="data.pkl"
if os.path.isfile(cachefile):
    with open(cachefile, "rb") as f:
        ds = pickle.load(f)
else:
    start = time.time()
    ds = BirdClefHarmonics(audio, fmin=500, fmax=4000)
    print(f"Loading data took {time.time()-start} seconds")
    with open(cachefile, "wb") as f:
        pickle.dump(ds, f)

In [5]:
loss = nn.CrossEntropyLoss(weight=torch.tensor(audio.label_weights()).to(training.DEVICE))

In [6]:
def accuracy(preds, y):
    return sum(preds.argmax(1) == y)

In [7]:
hps = training.HyperParameters(model_params={'n_labels': audio.n_labels, 'n_harmonics': 10, "num_filter_maps": 64, "kernel_size": 128}, 
                               optimizer_params={'lr': 1e-3})

In [8]:
results, model = training.train(data=ds, model_class = HarmonicModel, hyper_params=hps, loss_fn = loss,
                                name="convmodel", pad=True, batch_size=16, metrics={"Accuracy": accuracy})

Epoch   0: Loss=5.28259 val, 5.24245 train Accuracy=0.28571 val 0.28380 train 
Epoch   1: Loss=5.27685 val, 5.22527 train Accuracy=0.54342 val 0.42742 train 
Epoch   2: Loss=5.27080 val, 5.21229 train Accuracy=0.47619 val 0.44609 train 
Epoch   3: Loss=5.26244 val, 5.19938 train Accuracy=0.29132 val 0.27369 train 
Epoch   4: Loss=5.26171 val, 5.19378 train Accuracy=0.25210 val 0.30107 train 
Epoch   5: Loss=5.25417 val, 5.18623 train Accuracy=0.29692 val 0.29563 train 
Epoch   6: Loss=5.25815 val, 5.17914 train Accuracy=0.26891 val 0.26389 train 
Epoch   7: Loss=5.25504 val, 5.17413 train Accuracy=0.29692 val 0.28131 train 
Epoch   8: Loss=5.25571 val, 5.17155 train Accuracy=0.22409 val 0.30123 train 
Epoch   9: Loss=5.25443 val, 5.16826 train Accuracy=0.23529 val 0.29423 train 
Epoch  10: Loss=5.25496 val, 5.16239 train Accuracy=0.21289 val 0.25891 train 
Final results: Loss=5.254173755645752 Accuracy=0.29692 
